In [111]:
import maup # mggg's library for proration, see documentation here: https://github.com/mggg/maup
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
import os
import fiona
from statistics import mean, median
from pandas import read_csv
gp.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw' #To load KML files

# VEST Florida Validation

In [112]:
vest_fl_16 = gp.read_file("./raw-from-source/VEST/fl_2016/fl_2016.shp")

## Documentation

### Races

G16PRERTru - Donald J. Trump (Republican Party)  
G16PREDCli - Hillary Clinton (Democratic Party)  
G16PRELJoh - Gary Johnson (Libertarian Party)  
G16PRECCas - Darrell L. Castle (Constitution Party)  
G16PREGSte - Jill Stein (Green Party)  
G16PREIDeL - Roque De La Fuente (Reform Party)  
G16PREOth - Write-in Votes  

G16USSRRub - Marco Rubio (Republican Party)  
G16USSDMur - Patrick Murphy (Democratic Party)  
G16USSLSta - Paul Stanton (Libertarian Party)  
G16USSOth - Independent and Write-in Votes  

### Election Source

Election results from Florida Division of Elections (http://dos.myflorida.com/elections/data-statistics/elections-data/precinct-level-election-results/)  

### Precinct Source

The starting point was the Orlando Sentinel's precinct map for the 2016 presidential primary (http://interactive.orlandosentinel.com/elections/2016/presidential-primary/results/dem.html).  

Hillsborough, Lake, Miami-Dade, Orange, Palm Beach, and Pinellas Counties were updated directly with shapefiles downloaded from county Supervisor of Elections websites.  

Brevard, Marion, and Putnam Counties were updated with KML files from the respective county Supervisor of Elections websites, from their election result map pages.  

Columbia, DeSoto, Leon, Polk, and Osceola Counties updates received from Supervisor of Elections websites through personal contact.  

Highlands, Santa Rosa, Sarasota, and Volusia Counties had some precinct mergers, which were determined based on visual inspection of PDF precinct maps from the county websites.  

Charlotte, Hendry, Holmes, Jackson, Jefferson, Levy, Madison, and Walton Counties are from VTDs released by the Census Bureau's Redistricting Data Program.  

Hernando 99, Collier 450, and Palm Beach 8001/8002 are UOCAVA precincts with no real geography and votes were not redistributed.  

Precinct names were adjusted to align with the formatting used in the voter registration file (e.g., some precincts have leading spaces).  

In [113]:
#This Data is from the FL department of state, and can only be downloaded county by county
all_files = os.listdir("./raw-from-source/Election_Results/precinctlevelelectionresults2016gen")

In [114]:
#Check that all files have the same number of columns
for i in all_files:
    ref = "./raw-from-source/Election_Results/precinctlevelelectionresults2016gen/"
    file_ref = ref+i
    print(i)
    file_prev = pd.read_csv(file_ref,sep="\t",engine='python',index_col=None, header=None, error_bad_lines=False)
    print(file_prev.shape)
    
#All the files have 19 columns, so they should be good to combine

SEM_PctResults20161108.txt
(8154, 19)
STJ_PctResults20161108.txt
(4276, 19)
BAK_PctResults20161108.txt
(909, 19)
SAN_PctResults20161108.txt
(3415, 19)
MRN_PctResults20161108.txt
(12201, 19)
DIX_PctResults20161108.txt
(858, 19)
JEF_PctResults20161108.txt
(1206, 19)
PAS_PctResults20161108.txt
(13784, 19)
DES_PctResults20161108.txt
(1414, 19)
TAY_PctResults20161108.txt
(1111, 19)
GIL_PctResults20161108.txt
(709, 19)
SUW_PctResults20161108.txt


Skipping line 28: '	' expected after '"'
Skipping line 106: '	' expected after '"'
Skipping line 185: '	' expected after '"'
Skipping line 263: '	' expected after '"'
Skipping line 346: '	' expected after '"'
Skipping line 429: '	' expected after '"'
Skipping line 510: '	' expected after '"'
Skipping line 587: '	' expected after '"'
Skipping line 662: '	' expected after '"'
Skipping line 741: '	' expected after '"'
Skipping line 829: '	' expected after '"'
Skipping line 916: '	' expected after '"'
Skipping line 994: '	' expected after '"'
Skipping line 1076: '	' expected after '"'


(1556, 19)
GAD_PctResults20161108.txt
(2584, 19)
HER_PctResults20161108.txt
(3037, 19)
MON_PctResults20161108.txt
(3168, 19)
WAK_PctResults20161108.txt
(929, 19)
JAC_PctResults20161108.txt
(1230, 19)
OSC_PctResults20161108.txt
(9550, 19)
POL_PctResults20161108.txt
(21010, 19)
WAL_PctResults20161108.txt
(1722, 19)
HOL_PctResults20161108.txt
(898, 19)
HAR_PctResults20161108.txt
(1392, 19)
STL_PctResults20161108.txt
(6331, 19)
LEV_PctResults20161108.txt
(1001, 19)
CAL_PctResults20161108.txt
(898, 19)
ALA_PctResults20161108.txt
(6478, 19)
VOL_PctResults20161108.txt
(11871, 19)
DAD_PctResults20161108.txt
(69874, 19)
HIL_PctResults20161108.txt
(45890, 19)
CHA_PctResults20161108.txt
(7195, 19)
WAS_PctResults20161108.txt
(1664, 19)
ESC_PctResults20161108.txt
(8067, 19)
ORA_PctResults20161108.txt
(32989, 19)
CLA_PctResults20161108.txt
(4275, 19)
GLA_PctResults20161108.txt
(1044, 19)
HAM_PctResults20161108.txt
(755, 19)
LIB_PctResults20161108.txt
(856, 19)
SUM_PctResults20161108.txt
(2379, 19)
H

Skipping line 70: '	' expected after '"'
Skipping line 173: '	' expected after '"'
Skipping line 330: '	' expected after '"'
Skipping line 431: '	' expected after '"'
Skipping line 518: '	' expected after '"'
Skipping line 617: '	' expected after '"'
Skipping line 708: '	' expected after '"'
Skipping line 814: '	' expected after '"'
Skipping line 916: '	' expected after '"'
Skipping line 1014: '	' expected after '"'
Skipping line 1111: '	' expected after '"'
Skipping line 1210: '	' expected after '"'
Skipping line 1317: '	' expected after '"'
Skipping line 1421: '	' expected after '"'
Skipping line 1523: '	' expected after '"'
Skipping line 1628: '	' expected after '"'
Skipping line 1736: '	' expected after '"'
Skipping line 1831: '	' expected after '"'
Skipping line 1933: '	' expected after '"'
Skipping line 2025: '	' expected after '"'
Skipping line 2121: '	' expected after '"'
Skipping line 2218: '	' expected after '"'
Skipping line 2318: '	' expected after '"'
Skipping line 2422: '

Skipping line 27290: '	' expected after '"'
Skipping line 27385: '	' expected after '"'
Skipping line 27486: '	' expected after '"'
Skipping line 27577: '	' expected after '"'
Skipping line 27666: '	' expected after '"'
Skipping line 27763: '	' expected after '"'
Skipping line 27860: '	' expected after '"'
Skipping line 27952: '	' expected after '"'
Skipping line 28045: '	' expected after '"'
Skipping line 28140: '	' expected after '"'
Skipping line 28241: '	' expected after '"'
Skipping line 28338: '	' expected after '"'
Skipping line 28425: '	' expected after '"'
Skipping line 28523: '	' expected after '"'
Skipping line 28618: '	' expected after '"'
Skipping line 28718: '	' expected after '"'
Skipping line 28816: '	' expected after '"'
Skipping line 28910: '	' expected after '"'
Skipping line 29004: '	' expected after '"'
Skipping line 29093: '	' expected after '"'
Skipping line 29193: '	' expected after '"'
Skipping line 29295: '	' expected after '"'
Skipping line 29391: '	' expecte

Skipping line 48786: '	' expected after '"'
Skipping line 48900: '	' expected after '"'
Skipping line 48961: '	' expected after '"'
Skipping line 48962: '	' expected after '"'
Skipping line 49040: '	' expected after '"'
Skipping line 49185: '	' expected after '"'
Skipping line 49186: '	' expected after '"'
Skipping line 49301: '	' expected after '"'
Skipping line 49422: '	' expected after '"'
Skipping line 49423: '	' expected after '"'
Skipping line 49589: '	' expected after '"'
Skipping line 49675: '	' expected after '"'
Skipping line 49763: '	' expected after '"'
Skipping line 49855: '	' expected after '"'
Skipping line 49951: '	' expected after '"'
Skipping line 50048: '	' expected after '"'
Skipping line 50140: '	' expected after '"'
Skipping line 50236: '	' expected after '"'
Skipping line 50332: '	' expected after '"'
Skipping line 50428: '	' expected after '"'
Skipping line 50517: '	' expected after '"'
Skipping line 50608: '	' expected after '"'
Skipping line 50702: '	' expecte

(69962, 19)
OKA_PctResults20161108.txt
(4488, 19)
SAR_PctResults20161108.txt
(15496, 19)
CIT_PctResults20161108.txt
(2802, 19)
BRE_PctResults20161108.txt
(15505, 19)
BRO_PctResults20161108.txt
(69707, 19)
PIN_PctResults20161108.txt
(33620, 19)
MAD_PctResults20161108.txt
(1060, 19)
GUL_PctResults20161108.txt
(792, 19)
HIG_PctResults20161108.txt
(2856, 19)
MAN_PctResults20161108.txt
(7783, 19)
IND_PctResults20161108.txt
(6549, 19)


In [115]:
#Create a dataframe with the txt files
li = []
for i in all_files:
    ref = "./raw-from-source/Election_Results/precinctlevelelectionresults2016gen/"
    file_ref = ref+i
    file_prev = pd.read_csv(file_ref,sep="\t",engine='python',index_col=None, header=None,error_bad_lines=False)
    li.append(file_prev)
frame = pd.concat(li, axis=0, ignore_index=True)
print(frame.shape)

Skipping line 28: '	' expected after '"'
Skipping line 106: '	' expected after '"'
Skipping line 185: '	' expected after '"'
Skipping line 263: '	' expected after '"'
Skipping line 346: '	' expected after '"'
Skipping line 429: '	' expected after '"'
Skipping line 510: '	' expected after '"'
Skipping line 587: '	' expected after '"'
Skipping line 662: '	' expected after '"'
Skipping line 741: '	' expected after '"'
Skipping line 829: '	' expected after '"'
Skipping line 916: '	' expected after '"'
Skipping line 994: '	' expected after '"'
Skipping line 1076: '	' expected after '"'
Skipping line 70: '	' expected after '"'
Skipping line 173: '	' expected after '"'
Skipping line 330: '	' expected after '"'
Skipping line 431: '	' expected after '"'
Skipping line 518: '	' expected after '"'
Skipping line 617: '	' expected after '"'
Skipping line 708: '	' expected after '"'
Skipping line 814: '	' expected after '"'
Skipping line 916: '	' expected after '"'
Skipping line 1014: '	' expected af

Skipping line 36102: '	' expected after '"'
Skipping line 36189: '	' expected after '"'
Skipping line 36283: '	' expected after '"'
Skipping line 36364: '	' expected after '"'
Skipping line 36452: '	' expected after '"'
Skipping line 36541: '	' expected after '"'
Skipping line 36629: '	' expected after '"'
Skipping line 36719: '	' expected after '"'
Skipping line 36806: '	' expected after '"'
Skipping line 36896: '	' expected after '"'
Skipping line 36988: '	' expected after '"'
Skipping line 37064: '	' expected after '"'
Skipping line 37149: '	' expected after '"'
Skipping line 37241: '	' expected after '"'
Skipping line 37333: '	' expected after '"'
Skipping line 37421: '	' expected after '"'
Skipping line 37507: '	' expected after '"'
Skipping line 37598: '	' expected after '"'
Skipping line 37706: '	' expected after '"'
Skipping line 37707: '	' expected after '"'
Skipping line 37820: '	' expected after '"'
Skipping line 37908: '	' expected after '"'
Skipping line 37996: '	' expecte

Skipping line 59417: '	' expected after '"'
Skipping line 59495: '	' expected after '"'
Skipping line 59590: '	' expected after '"'
Skipping line 59669: '	' expected after '"'
Skipping line 59759: '	' expected after '"'
Skipping line 59835: '	' expected after '"'
Skipping line 59922: '	' expected after '"'
Skipping line 60018: '	' expected after '"'
Skipping line 60113: '	' expected after '"'
Skipping line 60203: '	' expected after '"'
Skipping line 60301: '	' expected after '"'
Skipping line 60398: '	' expected after '"'
Skipping line 60490: '	' expected after '"'
Skipping line 60585: '	' expected after '"'
Skipping line 60681: '	' expected after '"'
Skipping line 60741: '	' expected after '"'
Skipping line 60802: '	' expected after '"'
Skipping line 60881: '	' expected after '"'
Skipping line 61032: '	' expected after '"'
Skipping line 61033: '	' expected after '"'
Skipping line 61150: '	' expected after '"'
Skipping line 61246: '	' expected after '"'
Skipping line 61342: '	' expecte

(626229, 19)


In [116]:
frame.columns = frame.iloc[395275]
frame.drop([395275],inplace=True)
frame.reset_index(inplace=True,drop=True)
print(frame["CountyCode"].unique())
print(frame.shape)

['SEM' 'STJ' 'BAK' 'SAN' 'MRN' 'DIX' 'JEF' 'PAS' 'DES' 'TAY' 'GIL' 'SUW'
 'GAD' 'HER' 'MON' 'WAK' 'JAC' 'OSC' 'POL' 'WAL' 'HOL' 'HAR' 'STL' 'LEV'
 'CAL' 'ALA' 'VOL' 'DAD' 'HIL' 'CHA' 'WAS' 'ESC' 'ORA' 'CLA' 'GLA' 'HAM'
 'LIB' 'SUM' 'HEN' 'CLL' 'PUT' 'MRT' 'OKE' 'LEE' 'NAS' 'FLA' 'LEO' 'LAK'
 'CLM' 'DUV' 'BRA' 'UNI' 'FRA' 'BAY' 'LAF' 'IND' 'PAL' 'OKA' 'SAR' 'CIT'
 'BRE' 'BRO' 'PIN' 'MAD' 'GUL' 'HIG' 'MAN']
(626228, 19)


In [117]:
#Filter down to the relevant races
frame["ContestName"]=frame["ContestName"].str.upper()
office_List = ['PRESIDENT OF THE UNITED STATES','UNITED STATES SENATOR','President of the United States','United States Senator']
bad_name_List = ['OverVotes','UnderVotes','Times Blank Voted','Times Over Voted']
filtered_frame=frame[(frame["ContestName"].isin(office_List)) & ~(frame["CanName"].isin(bad_name_List))]
cand_name_dict = {"Trump / Pence":"Donald J. Trump","Clinton / Kaine":"Hillary R. Clinton",
                 "Johnson / Weld":"Gary Johnson","Stein / Baraka":"Jill Stein",
                 "Write-in":"WriteIn","De La Fuente / Steinberg":"Roque De La Fuente",
                 "Castle / Bradley":"Darrell L. Castle",'WriteInVotes':'WriteIn','WriteinVotes':'WriteIn'}
 
filtered_frame["CanName"]=filtered_frame["CanName"].map(cand_name_dict).fillna(filtered_frame["CanName"])
print(filtered_frame["CountyCode"].unique())

['SEM' 'STJ' 'BAK' 'SAN' 'MRN' 'DIX' 'JEF' 'PAS' 'DES' 'TAY' 'GIL' 'SUW'
 'GAD' 'HER' 'MON' 'WAK' 'JAC' 'OSC' 'POL' 'WAL' 'HOL' 'HAR' 'STL' 'LEV'
 'CAL' 'ALA' 'VOL' 'DAD' 'HIL' 'CHA' 'WAS' 'ESC' 'ORA' 'CLA' 'GLA' 'HAM'
 'LIB' 'SUM' 'HEN' 'CLL' 'PUT' 'MRT' 'OKE' 'LEE' 'NAS' 'FLA' 'LEO' 'LAK'
 'CLM' 'DUV' 'BRA' 'UNI' 'FRA' 'BAY' 'LAF' 'PAL' 'OKA' 'SAR' 'CIT' 'BRE'
 'BRO' 'PIN' 'MAD' 'GUL' 'HIG' 'MAN' 'IND']


<ipython-input-117-420cbeb3f689>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_frame["CanName"]=filtered_frame["CanName"].map(cand_name_dict).fillna(filtered_frame["CanName"])


In [118]:
#Make the precinct column at least 4 digits
filtered_frame["modified_pre"]=filtered_frame["Precinct"].astype(str).str.zfill(5) 


#Make a column with the 3 letter county code and the precincts
filtered_frame["Pct_std"]=filtered_frame["CountyCode"]+filtered_frame["modified_pre"]
print(filtered_frame.shape)

(85817, 21)


<ipython-input-118-4c9b250c6c14>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_frame["modified_pre"]=filtered_frame["Precinct"].astype(str).str.zfill(5)
<ipython-input-118-4c9b250c6c14>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_frame["Pct_std"]=filtered_frame["CountyCode"]+filtered_frame["modified_pre"]


In [119]:
print(filtered_frame["Pct_std"].str[0:3].unique())

['SEM' 'STJ' 'BAK' 'SAN' 'MRN' 'DIX' 'JEF' 'PAS' 'DES' 'TAY' 'GIL' 'SUW'
 'GAD' 'HER' 'MON' 'WAK' 'JAC' 'OSC' 'POL' 'WAL' 'HOL' 'HAR' 'STL' 'LEV'
 'CAL' 'ALA' 'VOL' 'DAD' 'HIL' 'CHA' 'WAS' 'ESC' 'ORA' 'CLA' 'GLA' 'HAM'
 'LIB' 'SUM' 'HEN' 'CLL' 'PUT' 'MRT' 'OKE' 'LEE' 'NAS' 'FLA' 'LEO' 'LAK'
 'CLM' 'DUV' 'BRA' 'UNI' 'FRA' 'BAY' 'LAF' 'PAL' 'OKA' 'SAR' 'CIT' 'BRE'
 'BRO' 'PIN' 'MAD' 'GUL' 'HIG' 'MAN' 'IND']


In [120]:
filtered_frame["CanName"]= filtered_frame["CanName"].str.upper()
filtered_frame["ContestName"] = filtered_frame["ContestName"].str.upper()
filtered_frame["pivot_col"]=filtered_frame["CanName"]+filtered_frame["ContestName"]

<ipython-input-120-385d14c6895e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_frame["CanName"]= filtered_frame["CanName"].str.upper()
<ipython-input-120-385d14c6895e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_frame["ContestName"] = filtered_frame["ContestName"].str.upper()
<ipython-input-120-385d14c6895e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [121]:
pivoted_2016 = pd.pivot_table(filtered_frame, values=["CanVotes"], index=["Pct_std"],columns=["pivot_col"],aggfunc=sum)
pivoted_2016.columns = pivoted_2016.columns.droplevel(0)
pivoted_2016.reset_index(drop=False,inplace=True)
pivoted_2016.shape
print(len(pivoted_2016.columns))
print(pivoted_2016.columns)

16
Index(['Pct_std', 'BASIL E. DALACKUNITED STATES SENATOR',
       'BRUCE NATHANUNITED STATES SENATOR',
       'DARRELL L. CASTLEPRESIDENT OF THE UNITED STATES',
       'DONALD J. TRUMPPRESIDENT OF THE UNITED STATES',
       'GARY JOHNSONPRESIDENT OF THE UNITED STATES',
       'HILLARY R. CLINTONPRESIDENT OF THE UNITED STATES',
       'JILL STEINPRESIDENT OF THE UNITED STATES',
       'MARCO RUBIOUNITED STATES SENATOR',
       'PATRICK MURPHYUNITED STATES SENATOR',
       'PAUL STANTONUNITED STATES SENATOR',
       'ROQUE DE LA FUENTEPRESIDENT OF THE UNITED STATES',
       'STEVEN MACHATUNITED STATES SENATOR',
       'TONY KHOURYUNITED STATES SENATOR',
       'WRITEINPRESIDENT OF THE UNITED STATES',
       'WRITEINUNITED STATES SENATOR'],
      dtype='object', name='pivot_col')


In [122]:
pivoted_2016['Pct_std'].str[0:3].unique()

array(['ALA', 'BAK', 'BAY', 'BRA', 'BRE', 'BRO', 'CAL', 'CHA', 'CIT',
       'CLA', 'CLL', 'CLM', 'DAD', 'DES', 'DIX', 'DUV', 'ESC', 'FLA',
       'FRA', 'GAD', 'GIL', 'GLA', 'GUL', 'HAM', 'HAR', 'HEN', 'HER',
       'HIG', 'HIL', 'HOL', 'IND', 'JAC', 'JEF', 'LAF', 'LAK', 'LEE',
       'LEO', 'LEV', 'LIB', 'MAD', 'MAN', 'MON', 'MRN', 'MRT', 'NAS',
       'OKA', 'OKE', 'ORA', 'OSC', 'PAL', 'PAS', 'PIN', 'POL', 'PUT',
       'SAN', 'SAR', 'SEM', 'STJ', 'STL', 'SUM', 'SUW', 'TAY', 'UNI',
       'VOL', 'WAK', 'WAL', 'WAS'], dtype=object)

In [123]:
pivoted_2016.columns=(['countypct',"sen_wi_1",
                       "sen_wi_2",
                       "G16PRECCas",
                       "G16PRERTru",
                       "G16PRELJon",
                        "G16PREDCli",
                       "G16PREGSte",
                       "G16USSRRub",
                       "G16USSDMur",
                       "G16USSLSta",
                       "G16PREIDeL",
                       "sen_wi_3",
                       "sen_wi_4",  
                       "G16PREoth",
                       "sen_wi_5"])
print(pivoted_2016.columns)
print(pivoted_2016.shape)
pivoted_2016["G16USSOth"]=pivoted_2016["sen_wi_1"]+pivoted_2016["sen_wi_2"]+pivoted_2016["sen_wi_3"]+pivoted_2016["sen_wi_4"]+pivoted_2016["sen_wi_5"]
pivoted_2016.drop(["sen_wi_1","sen_wi_2","sen_wi_3","sen_wi_4","sen_wi_5"], axis=1,inplace=True)

Index(['countypct', 'sen_wi_1', 'sen_wi_2', 'G16PRECCas', 'G16PRERTru',
       'G16PRELJon', 'G16PREDCli', 'G16PREGSte', 'G16USSRRub', 'G16USSDMur',
       'G16USSLSta', 'G16PREIDeL', 'sen_wi_3', 'sen_wi_4', 'G16PREoth',
       'sen_wi_5'],
      dtype='object')
(5870, 16)


In [124]:
pivoted_2016 = pivoted_2016.fillna(0)
print(sum(pivoted_2016["G16PREDCli"]))
pivoted_2016["county"]=pivoted_2016["countypct"].str[0:3]

4508731.0


In [125]:
print(sum(vest_fl_16["G16PREDCli"]))

4504158


In [126]:
vest_fl_16.groupby(["county"]).sum()

,G16PRERTru,G16PREDCli,G16PRELJon,G16PRECCas,G16PREGSte,G16PREIDeL,G16PREoth,G16USSRRub,G16USSDMur,G16USSLSta,G16USSOth
county,,,,,,,,,,,
ALA,46834,75820,4059,262,1507,87,1531,54203,69399,2294,1642
BAK,10294,2112,169,23,30,6,71,9901,2135,232,218
BAY,62194,21797,2652,179,562,65,773,62906,19971,2484,1498
BRA,8913,2924,177,33,47,4,60,8720,2694,232,235
BRE,181848,119679,9451,744,2708,314,3397,181496,112914,9709,7882
...,...,...,...,...,...,...,...,...,...,...,...
UNI,4568,1014,90,7,12,3,28,4313,1064,106,106
VOL,143007,109091,6111,440,2003,208,2418,139933,104531,7395,5257
WAK,10512,4348,378,32,66,14,94,9773,4852,330,256


In [127]:
pd.set_option('display.max_rows', 70)
display(pivoted_2016.groupby(["county"]).sum()["G16PREDCli"]-vest_fl_16.groupby(["county"]).sum()["G16PREDCli"])

county
ALA       0.0
BAK       0.0
BAY       0.0
BRA       0.0
BRE       0.0
BRO       0.0
CAL       0.0
CHA       0.0
CIT       0.0
CLA       0.0
CLL      72.0
CLM       0.0
DAD       0.0
DES       0.0
DIX       0.0
DUV       0.0
ESC       0.0
FLA       9.0
FRA       0.0
GAD       0.0
GIL       0.0
GLA       0.0
GUL       0.0
HAM       0.0
HAR       0.0
HEN       0.0
HER       0.0
HIG       0.0
HIL       0.0
HOL       0.0
IND       0.0
JAC       0.0
JEF       0.0
LAF       0.0
LAK       5.0
LEE       0.0
LEO       0.0
LEV       0.0
LIB       0.0
MAD       0.0
MAN       0.0
MON       0.0
MRN       0.0
MRT       0.0
NAS       0.0
OKA       0.0
OKE       0.0
ORA       0.0
OSC      52.0
PAL    4435.0
PAS       0.0
PIN       0.0
POL       0.0
PUT       0.0
SAN       0.0
SAR       0.0
SEM       0.0
STJ       0.0
STL       0.0
SUM       0.0
SUW       0.0
TAY       0.0
UNI       0.0
VOL       0.0
WAK       0.0
WAL       0.0
WAS       0.0
Name: G16PREDCli, dtype: float64

In [128]:
print(vest_fl_16.head())
print(pivoted_2016.head())

   pct county countypct  G16PRERTru  G16PREDCli  G16PRELJon  G16PRECCas  \
0  001    DAD    DAD001         277         195           4           0   
1  010    DAD    DAD010          20          16           0           0   
2  100    DAD    DAD100         641        2893          66          17   
3  101    DAD    DAD101         679        1096          34           0   
4  102    DAD    DAD102        1100        1596          29           1   

   G16PREGSte  G16PREIDeL  G16PREoth  G16USSRRub  G16USSDMur  G16USSLSta  \
0           2           0          5         337         133           3   
1           0           0          0          30           6           0   
2          41          16         24         897        2434          50   
3           7           0         18         845         947          14   
4           4           2         22        1399        1274          16   

   G16USSOth                                           geometry  
0          5  POLYGON Z ((

In [129]:
print(pivoted_2016.columns)

Index(['countypct', 'G16PRECCas', 'G16PRERTru', 'G16PRELJon', 'G16PREDCli',
       'G16PREGSte', 'G16USSRRub', 'G16USSDMur', 'G16USSLSta', 'G16PREIDeL',
       'G16PREoth', 'G16USSOth', 'county'],
      dtype='object')


In [130]:
vest_fl_16["countypct"] = vest_fl_16["county"]+vest_fl_16["pct"].str.zfill(5)
print(vest_fl_16.head())

   pct county countypct  G16PRERTru  G16PREDCli  G16PRELJon  G16PRECCas  \
0  001    DAD  DAD00001         277         195           4           0   
1  010    DAD  DAD00010          20          16           0           0   
2  100    DAD  DAD00100         641        2893          66          17   
3  101    DAD  DAD00101         679        1096          34           0   
4  102    DAD  DAD00102        1100        1596          29           1   

   G16PREGSte  G16PREIDeL  G16PREoth  G16USSRRub  G16USSDMur  G16USSLSta  \
0           2           0          5         337         133           3   
1           0           0          0          30           6           0   
2          41          16         24         897        2434          50   
3           7           0         18         845         947          14   
4           4           2         22        1399        1274          16   

   G16USSOth                                           geometry  
0          5  POLYGON Z ((

In [131]:
source_precinct_election_changes_dict = {"WAS00014":"WAS00017","WAS00016":"WAS00019","HAR00017":"HAR00009","HAR00015":"HAR00007",
    "PUT00031":"PUT00035","HAR00018":"HAR00010"}

pivoted_2016["countypct"]=pivoted_2016["countypct"].map(source_precinct_election_changes_dict).fillna(pivoted_2016["countypct"])


In [132]:
#Deal with 0 votes precincts
vest_empty = vest_fl_16[(vest_fl_16['G16PRERTru']==0) & (vest_fl_16['G16PREDCli']==0) &
              (vest_fl_16['G16PRELJon']==0) & (vest_fl_16['G16PRECCas']==0) &
              (vest_fl_16['G16PREGSte']==0) & (vest_fl_16['G16PREIDeL']==0) &
              (vest_fl_16['G16PREoth']==0) & (vest_fl_16['G16USSRRub']==0) &
              (vest_fl_16['G16USSDMur']==0) & (vest_fl_16['G16USSLSta']==0) &
              (vest_fl_16['G16USSOth']==0)]

#Create the zero vote precincts in the MEDSL file

new_precincts = pd.DataFrame(columns=pivoted_2016.columns)
new_precincts["countypct"]=vest_empty["countypct"]
new_precincts["county"]=vest_empty["countypct"].str[0:3]

col_list = ['G16PRECCas', 'G16PRERTru', 'G16PRELJon', 'G16PREDCli',
       'G16PREGSte', 'G16USSRRub', 'G16USSDMur', 'G16USSLSta', 'G16PREIDeL',
       'G16PREoth', 'G16USSOth']
for i in col_list:
    new_precincts[i]=0

print(new_precincts.head())

pivoted_2016= pivoted_2016.append(new_precincts)
pivoted_2016.reset_index(drop=True,inplace=True)
print(pivoted_2016.head())

    countypct  G16PRECCas  G16PRERTru  G16PRELJon  G16PREDCli  G16PREGSte  \
273  DAD00383           0           0           0           0           0   
281  DAD00391           0           0           0           0           0   
282  DAD00392           0           0           0           0           0   
283  DAD00394           0           0           0           0           0   
284  DAD00395           0           0           0           0           0   

     G16USSRRub  G16USSDMur  G16USSLSta  G16PREIDeL  G16PREoth  G16USSOth  \
273           0           0           0           0          0          0   
281           0           0           0           0          0          0   
282           0           0           0           0          0          0   
283           0           0           0           0          0          0   
284           0           0           0           0          0          0   

    county  
273    DAD  
281    DAD  
282    DAD  
283    DAD  
284    DA

In [133]:
check_election = pd.merge(vest_fl_16,pivoted_2016,how="outer",on="countypct",indicator=True)

print(check_election["_merge"].value_counts())
check_election[check_election["_merge"]=="left_only"].to_csv("./vest_election.csv")
check_election[check_election["_merge"]=="right_only"].to_csv("./source_election.csv")

both          5983
right_only      23
left_only       17
Name: _merge, dtype: int64


In [134]:
print(vest_fl_16[vest_fl_16["countypct"]=="UNI0001A"])

    pct county countypct  G16PRERTru  G16PREDCli  G16PRELJon  G16PRECCas  \
908  1A    UNI  UNI0001A         236          56           5           0   

     G16PREGSte  G16PREIDeL  G16PREoth  G16USSRRub  G16USSDMur  G16USSLSta  \
908           1           0          2         228          62           2   

     G16USSOth                                           geometry  
908          5  POLYGON Z ((234533.388 2067718.870 0.000, 2345...  


In [135]:
merge these precincts:
UNI0001A
UNI0001B
UNI0002A
UNI0002B
UNI0003A
UNI0003B
UNI0004A
UNI0004C
UNI0005A
UNI0005C

call them this:
UNI1A & 1B
UNI2A & 2B
UNI3A & 3B
UNI4A & 4C
UNI5A & 5C   


election_precinct_merger(["UNI0001A","UNI0001B"],"UNI1A & 1B")
election_precinct_merger(["UNI0002A","UNI0002B"],"UNI2A & 2B")
election_precinct_merger(["UNI0003A","UNI0003B"],"UNI3A & 3B")
election_precinct_merger(["UNI0004A","UNI0004C"],"UNI4A & 4C")
election_precinct_merger(["UNI0005A","UNI0005C"],"UNI5A & 5C")

SyntaxError: invalid syntax (<ipython-input-135-1a8d5de1004a>, line 1)

In [136]:
print(check_election[check_election["_merge"]=="both"])

      pct county_x countypct  G16PRERTru_x  G16PREDCli_x  G16PRELJon_x  \
0     001      DAD  DAD00001         277.0         195.0           4.0   
1     010      DAD  DAD00010          20.0          16.0           0.0   
2     100      DAD  DAD00100         641.0        2893.0          66.0   
3     101      DAD  DAD00101         679.0        1096.0          34.0   
4     102      DAD  DAD00102        1100.0        1596.0          29.0   
...   ...      ...       ...           ...           ...           ...   
5995    4      HOL  HOL00004        1077.0         105.0          24.0   
5996    5      HOL  HOL00005         390.0          30.0           6.0   
5997    6      HOL  HOL00006        1667.0         126.0          26.0   
5998    7      HOL  HOL00007        1307.0         180.0          26.0   
5999    8      HOL  HOL00008         835.0         151.0          18.0   

      G16PRECCas_x  G16PREGSte_x  G16PREIDeL_x  G16PREoth_x  ...  \
0              0.0           2.0           

In [27]:
print(sum(vest_fl_16["G16USSOth"]))
print(sum(pivoted_2016["G16USSOth"]))

168360
155700.0


In [28]:
print(pivoted_2016.columns)
print(vest_fl_16.columns)

Index(['countypct', 'G16PRECCas', 'G16PRERTru', 'G16PRELJon', 'G16PREDCli',
       'G16PREGSte', 'G16USSRRub', 'G16USSDMur', 'G16USSLSta', 'G16PREIDeL',
       'G16PREoth', 'G16USSOth', 'county'],
      dtype='object')
Index(['pct', 'county', 'countypct', 'G16PRERTru', 'G16PREDCli', 'G16PRELJon',
       'G16PRECCas', 'G16PREGSte', 'G16PREIDeL', 'G16PREoth', 'G16USSRRub',
       'G16USSDMur', 'G16USSLSta', 'G16USSOth', 'geometry'],
      dtype='object')


In [29]:
def validater_row (df, column_List):
    matching_rows = 0
    different_rows = 0
    diff_list=[]
    diff_values = []
    max_diff = 0
    
    for j in range(0,len(df.index)):
        same = True
        for i in column_List:
            left_Data = i + "_x"
            right_Data = i + "_y"
            diff = abs(df.iloc[j][left_Data]-df.iloc[j][right_Data])
            if(diff != 0):
                print(df.iloc[j]['countypct'])
                print(i)
                diff_values.append(abs(diff))
                same = False
                if(np.isnan(diff)):
                    print("NaN value at diff is: ", df.iloc[j]['countypct'])
                    print(df.iloc[j][left_Data])
                    print(df.iloc[j][right_Data])
                if (diff>max_diff):
                    max_diff = diff
                    print("New max diff is: ", str(max_diff))
                    print(df.iloc[j]['countypct'])
        if(same != True):
            different_rows +=1
            diff_list.append(df.iloc[j]['countypct'])
        else:
            matching_rows +=1
    print("There are ", len(df.index)," total rows")
    print(different_rows," of these rows have election result differences")
    print(matching_rows," of these rows are the same")
    print("")
    print("The max difference between any one shared column in a row is: ", max_diff)
    if(len(diff_values)!=0):
        print("The average difference is: ", str(sum(diff_values)/len(diff_values)))
    count_big_diff = len([i for i in diff_values if i > 10])
    print("There are ", str(count_big_diff), "precinct results with a difference greater than 5")
    diff_list.sort()
    print(diff_list)

col_list = ['G16PRECCas', 'G16PRERTru', 'G16PRELJon', 'G16PREDCli',
       'G16PREGSte', 'G16USSRRub', 'G16USSDMur', 'G16USSLSta', 'G16PREIDeL']

#Took out Senate other and president other

validater_row(check_election[check_election["_merge"]=="both"],col_list)

SAR00429
G16PRERTru
New max diff is:  683.0
SAR00429
SAR00429
G16PRELJon
SAR00429
G16PREDCli
SAR00429
G16PREGSte
SAR00429
G16USSRRub
New max diff is:  729.0
SAR00429
SAR00429
G16USSDMur
SAR00429
G16USSLSta
SAR00501
G16PRERTru
New max diff is:  1603.0
SAR00501
SAR00501
G16PRELJon
SAR00501
G16PREDCli
SAR00501
G16PREGSte
SAR00501
G16USSRRub
New max diff is:  1622.0
SAR00501
SAR00501
G16USSDMur
SAR00501
G16USSLSta
SAR00501
G16PREIDeL
PUT00021
G16PRECCas
PUT00021
G16PRERTru
PUT00021
G16PRELJon
PUT00021
G16PREDCli
PUT00021
G16PREGSte
PUT00021
G16USSRRub
PUT00021
G16USSDMur
PUT00021
G16USSLSta
PUT00021
G16PREIDeL
PUT00020
G16PRERTru
PUT00020
G16PRELJon
PUT00020
G16PREDCli
PUT00020
G16PREGSte
PUT00020
G16USSRRub
PUT00020
G16USSDMur
PUT00020
G16USSLSta
PUT00020
G16PREIDeL
PUT00018
G16PRECCas
PUT00018
G16PRERTru
PUT00018
G16PRELJon
PUT00018
G16PREDCli
PUT00018
G16PREGSte
PUT00018
G16USSRRub
PUT00018
G16USSDMur
PUT00018
G16USSLSta
PUT00018
G16PREIDeL
PUT00019
G16PRECCas
PUT00019
G16PRERTru
PUT000

In [30]:
diff_list = ['HAR00005', 'HAR00008', 'HAR00011', 'HAR00012', 'PAL01173', 'PAL01189', 'PAL01247', 'PAL02081', 'PAL02083', 'PAL02097', 'PAL02116', 'PAL02126', 'PAL04129', 'PAL05003', 'PAL05007', 'PAL05018', 'PAL05055', 'PAL05112', 'PAL05113', 'PAL05115', 'PAL05117', 'PAL05119', 'PAL06207', 'PAL07149', 'PUT00014', 'PUT00015', 'PUT00016', 'PUT00018', 'PUT00019', 'PUT00020', 'PUT00021', 'PUT00022', 'PUT00023', 'PUT00024', 'PUT00025', 'PUT00026', 'PUT00027', 'PUT00028', 'PUT00030', 'PUT00032', 'PUT00033', 'PUT00036', 'PUT00037', 'SAR00429', 'SAR00501', 'WAL00430', 'WAL00440', 'WAL00540', 'WAS00015']

In [31]:
print(check_election[check_election["_merge"]=="left_only"]["countypct"])

922     UNI0001A
923     UNI0001B
924     UNI0002A
925     UNI0002B
926     UNI0003A
927     UNI0003B
929     UNI0004A
931     UNI0004C
932     UNI0005A
934     UNI0005C
1089    PUT00034
1098    PUT00038
1109    PUT00039
5123    HAR00003
5124    HAR00004
5126    HAR00006
5771    WAS00018
Name: countypct, dtype: object


In [32]:
pd.set_option('display.max_columns', 500)
display(check_election[check_election["countypct"]=='WAL00420'])

,pct,county_x,countypct,G16PRERTru_x,G16PREDCli_x,G16PRELJon_x,G16PRECCas_x,G16PREGSte_x,G16PREIDeL_x,G16PREoth_x,G16USSRRub_x,G16USSDMur_x,G16USSLSta_x,G16USSOth_x,geometry,G16PRECCas_y,G16PRERTru_y,G16PRELJon_y,G16PREDCli_y,G16PREGSte_y,G16USSRRub_y,G16USSDMur_y,G16USSLSta_y,G16PREIDeL_y,G16PREoth_y,G16USSOth_y,county_y,_merge
5979,420,WAL,WAL00420,381.0,68.0,10.0,0.0,1.0,0.0,0.0,379.0,53.0,8.0,11.0,"POLYGON Z ((-1012400.981 2271653.512 0.000, -1...",0.0,381.0,10.0,68.0,1.0,379.0,53.0,8.0,0.0,0.0,0.0,WAL,both


In [138]:
print(len(vest_fl_16["county"].unique()))

67


### Shapefiles

The starting point was the Orlando Sentinel's precinct map for the 2016 presidential primary (http://interactive.orlandosentinel.com/elections/2016/presidential-primary/results/dem.html).  

Hillsborough, Lake, Miami-Dade, Orange, Palm Beach, and Pinellas Counties were updated directly with shapefiles downloaded from county Supervisor of Elections websites.  

Brevard, Marion, and Putnam Counties were updated with KML files from the respective county Supervisor of Elections websites, from their election result map pages.  

Columbia, DeSoto, Leon, Polk, and Osceola Counties updates received from Supervisor of Elections websites through personal contact.  

Highlands, Santa Rosa, Sarasota, and Volusia Counties had some precinct mergers, which were determined based on visual inspection of PDF precinct maps from the county websites.  

Charlotte, Hendry, Holmes, Jackson, Jefferson, Levy, Madison, and Walton Counties are from VTDs released by the Census Bureau's Redistricting Data Program.  

Hernando 99, Collier 450, and Palm Beach 8001/8002 are UOCAVA precincts with no real geography and votes were not redistributed.  

Precinct names were adjusted to align with the formatting used in the voter registration file (e.g., some precincts have leading spaces).  

In [ ]:
#The below had shapefiles, but they were not able to load:
DAD,HIG,LAK,LEE,PAL,PIN,SUM

#Multiple files for BAY, CLL, SUM, SUW, VOL

For some of these files, I either exported a XlS file to csv to make it easier to load or 

In [139]:
GAD_pct=pd.read_csv("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/GAD/deleted_columns_PctBlock.csv",error_bad_lines=False)
#print(GAD_pct.head())

GIL_pct=pd.read_csv("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/GIL/GIL20121106v6_PctBlock.txt",sep="\t", header=None)
GIL_pct.reset_index(inplace=True,drop=True)
GIL_pct = GIL_pct[[0,1,2]]
GIL_pct.columns = ['County','Precinct','Block']
#print(GIL_pct.head())

HAM_pct=pd.read_csv("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/HAM/HAM20140707_PctBlocka8ac771f-a98a-4718-bb62-f6dceab1369c.txt",sep="\t", header=None)
HAM_pct.reset_index(inplace=True,drop=True)
HAM_pct = HAM_pct[[0,1,2]]
HAM_pct.columns = ['County','Precinct','Block']
#print(HAM_pct.head())

JAC_pct=pd.read_csv("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/JAC/cleaned_PctBlock.csv")
JAC_pct.columns = ['County','Precinct','Block']
JAC_pct["County"]="JAC"
#print(JAC_pct.head())

MAD_pct=pd.read_csv("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/MAD/clean_PctBlock.csv")
MAD_pct.columns = ['County','Precinct','Block']
#print(MAD_pct.head())

MRN_pct=pd.read_csv("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/MRN/MRN20160510PctBlocka03accb3-5931-4025-b0b3-ee84b1d78dce.txt",sep="\t", header=None)
MRN_pct.columns = ['County','Precinct','Block']
#print(MRN_pct.head())

TAY_pct=pd.read_csv("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/TAY/TAY20120806_PctBlock0b8117f6-c1de-4b3f-8c25-1e78e2013cd7.txt",sep="\t", header=None)
TAY_pct.columns = ['County','Precinct','Block']
#print(TAY_pct.head())

DAD_pct=pd.read_csv("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/DAD/DAD20150817_PctBlock8708ddd9-66e2-42f4-8249-d4aa549f0d92.txt",sep="\t")
DAD_pct=DAD_pct[["COUNTY CODE","PRECINCT NUMBER","CENSUS BLOCK"]]
DAD_pct.columns = ['County','Precinct','Block']
#print(DAD_pct.head())

HIG_pct=pd.read_csv("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/HIG/HIG20160801_PctBlock2b83ba25-1180-477a-b954-b31f1605fd92.txt",sep="\t")
HIG_pct=HIG_pct[["County Code","Precinct Number","Census Block"]]
HIG_pct.columns = ['County','Precinct','Block']
HIG_pct["County"]="HIG"
#print(HIG_pct.head())

LAK_pct=pd.read_csv("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/LAK/LAK20160420_PctBlock73e1ca5e-e9f7-48e1-b380-98b8401ad0f0.txt",sep="\t")
LAK_pct=LAK_pct[["County Code","Precinct Number","Census Block"]]
LAK_pct.columns = ['County','Precinct','Block']
LAK_pct["County"]="LAK"
#print(LAK_pct.head())

LEE_pct=pd.read_csv("/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pdv/pdv-fl/vest-fl-2016/raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/LEE/LEE20180813_PctBlock495f9750-05d2-4aa6-9b20-72a6a1fcef02.txt",sep="\t",header=None)
LEE_pct["County"]="LEE"
LEE_pct = LEE_pct[[1,2,'County']]
LEE_pct.columns = ['Precinct','Block','County']
LEE_pct = LEE_pct[['County','Precinct','Block']]
#print(LEE_pct.head())

PIN_pct=pd.read_csv("/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pdv/pdv-fl/vest-fl-2016/raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/PIN/PIN20160503PCTBLOCK6239b364-d072-49e8-a23f-5162bcb5e8f7.TXT",sep=",")
PIN_pct["County"]="PIN"
PIN_pct.columns = ['Block','Precinct','County']
PIN_pct = PIN_pct[['County','Precinct','Block']]
#print(PIN_pct.head())

SUM_pct=pd.read_csv("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/SUM/SUM20150414_PctBlock5aacc620-010f-48b7-b03e-074bc85a3d2a.txt",sep='\t')
SUM_pct = SUM_pct[['County Code','PrecinctNu','CensusBloc']]
SUM_pct.columns = ['County','Precinct','Block']
SUM_pct["County"]="SUM"
#print(SUM_pct.head())

In [100]:
block_file_list = [GAD_pct,GIL_pct,HAM_pct,JAC_pct,MAD_pct,MRN_pct,TAY_pct,DAD_pct,HIG_pct,LAK_pct,LEE_pct,PIN_pct,SUM_pct]
block_counties = pd.concat(block_file_list)
block_counties["unique_ID"]=block_counties["County"]+block_counties["Precinct"].astype(str)


['GAD' 'GIL' 'HAM' 'JAC' 'MAD' 'MRN' 'TAY' 'DAD' 'HIG' 'LAK' 'LEE' 'PIN'
 'SUM']


  County Precinct            Block unique_ID
0    GAD        1  120390205001012      GAD1
1    GAD        1  120390205001013      GAD1
2    GAD        1  120390205001016      GAD1
3    GAD        1  120390205001017      GAD1
4    GAD        1  120390205001018      GAD1


In [105]:
#Shapefiles built out of block assignments
fl_2010_census_blocks = gp.read_file("./raw-from-source/Census_Blocks/fl_2010_b_bound/fl_2010_b_bound.shp")
fl_2010_census_blocks.rename(columns={"GEOID": "Block"},inplace=True)
fl_2010_census_blocks["Block"] = fl_2010_census_blocks["Block"].astype(int)
fl_2010_census_blocks = fl_2010_census_blocks[["Block","geometry"]]
block_counties_merge = pd.merge(block_counties,fl_2010_census_blocks,on="Block",how="left")
block_counties_merge=gp.GeoDataFrame(block_counties_merge)
census_block_precincts = block_counties_merge.dissolve(by="unique_ID")

In [141]:
census_block_precincts.reset_index(inplace=True,drop=True)
print(census_block_precincts.head())

                                            geometry County Precinct  \
0  POLYGON ((-80.11869 25.95714, -80.11992 25.957...    DAD        1   
1  POLYGON ((-80.13581 25.87200, -80.13810 25.871...    DAD       10   
2  POLYGON ((-80.35118 25.80440, -80.35095 25.804...    DAD      100   
3  POLYGON ((-80.15749 25.96307, -80.15965 25.963...    DAD      101   
4  POLYGON ((-80.14698 25.95983, -80.14705 25.961...    DAD      102   

             Block  
0  120860001152001  
1  120860001202023  
2  120860090103018  
3  120860097031000  
4  120860001331001  


In [150]:
print(len(census_block_precincts["County"].unique()))
print(census_block_precincts.shape)

13
(1597, 4)


In [224]:
ALA_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/ALA/ALA20121106v6_PctMap/ALA20121106v6_PctMap.shp")
BAK_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/BAK/BAK20121106v6_PctMap/BAK20121106v6_PctMap.shp")
BAY_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/BAK/BAK20121106v6_PctMap/BAK20121106v6_PctMap.shp")
BRA_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/BRA/BRA20121106v6_PctMap/BRA20121106v5_PctMap.shp")
BRE_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/BRE/Post RD Precincts/Precincts_2016.shp")
BRO_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/BRO/PRECINCTS/PRECINCTS_region.shp")
CAL_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/CAL/CAL20121106v6_PctMap/CAL20121106v6_PctMap.shp")
CHA_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/CHA/CHA20121106v6_PctMap/CHA20121106v5_PctMap.shp")
CIT_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/CIT/CIT20170531_PctMap/PRECINCT05312017.shp")
CLA_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/CLA/CLA20160503_PctMapcc84b5a6-3ade-4dd9-acd8-4c8157bcd343/CLA20160503_PctMap.shp")
CLA_pct = CLA_pct.dissolve(by="PRECINCT")
CLA_pct.reset_index(inplace=True)
CLL_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/CLL/Shape Files/PRECINCT12021.shp")
CLM_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/CLM/CLM20121106v6_PctMap/CLM20121106v6_PctMap.shp")
DES_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/DES/DES20121106v6_PctMap/DES20121106v5_PctMap.shp")
DIX_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/DIX/DIX20121106v6_PctMap/DIX20121106v6_PctMap.shp")
DUV_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/DUV/DUV20120615_PctMap_region38f2bdfc-7be4-466e-ab20-98b2ad332567/DUV20120615_PctMap_region.shp")
ESC_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/ESC/Precinct_Map_2018/PRECINCT_MAP_022018.shp")
FLA_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/FLA/FLA20121106v6_PctMap/FLA20121106v6_PctMap.shp")
FRA_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/FRA/FRA20121106v6_PctMap/FRA20121106v6_PctMap.shp")
GLA_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/GLA/GLA20121106v6_PctMap/GLA20121106v6_PctMap.shp")
GUL_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/GUL/GUL20121106v6_PctMap/GUL20121106v6_PctMap.shp")
HAR_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/HAR/HAR20121106v6_PctMap/HAR20121106v6_PctMap.shp")
HEN_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/HEN/HEN20121106v6_PctMaped1303a5-dc65-41db-ac5f-a2bc3e07721a/HEN20121106v5_PctMap.shp")
HER_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/HER/HER20121106v6_PctMap/HER20121106v6_PctMap.shp")
HIL_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/HIL/HIL20170620_PctMap9febfa30-eb46-4b2b-98d8-efb4bf232874/PctMap.shp")
HOL_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/HOL/HOL20121106v6_PctMap/HOL20121106v6_PctMap.shp")
IND_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/IND/IND20121106v6_PctMap/IND20121106v6_PctMap.shp")
JEF_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/JEF/JEF20121106v6_PctMap/JEF20121106v6_PctMap.shp")
LAF_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/LAF/LAF20121106v6_PctMap/LAF20121106v6_PctMap.shp")
LEO_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/LEO/LEO20160608_PctMapf23ef83b-963f-4d2a-a377-6832ef66b253/LEO02160608_PctMap.shp")
LEV_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/LEV/LEV20121106v6_PctMap/LEV20121106v6_PctMap.shp")
LIB_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/LIB/LIB20121106v6_PctMap/LIB20121106v5_PctMap.shp")
MAN_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/MAN/MAN20121106v6_PctMap/MAN20121106v6_PctMap.shp")
MON_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/MON/MON20121106v6_PctMap/MON20121106v6_PctMap.shp")
MRT_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/MRT/MRT20121106v6_PctMap/MRT20121106v6_PctMap.shp")
NAS_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/NAS/NAS20121106v6_PctMap/NAS20121106v5_PctMap.shp")
OKA_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/OKA/OKA20121106_PCTv6da58ba66-dc93-4a57-b652-54e0178dd8c0/OKA20121106v6_PctMap/OKA20121106v5_PctMap.shp")
OKE_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/OKE/OKE20120628_PctMap515d73e7-c7f9-43ac-83df-e27de5e49af7/2012 Precincts.shp")
ORA_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/ORA/ORA20160506_PctMapc9f3a643-0427-4f76-83e6-015be0bd518c/ORA20160506_PctMap_region.shp")
OSC_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/OSC/OSC20160614_PctMapee50a15b-49e5-40e9-b975-fc25e040399a/2016_06 Osceola Precincts.shp")
PAL_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/PAL/SHAPEFILES 2012_region/SHAPEFILES 2012_region.shp")
PAS_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/PAS/ESRI Shapefiles/Pasco.shp")
POL_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/POL/POL20160511_PctMapbb417d62-028d-4b78-adbf-344f474213b2/2016 Polk Precincts.shp")
PUT_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/PUT/PUT20160614PctMap98164384-dc27-48c1-8ae9-dfa69c6afdc0/PUT20160614PctMap.shp")
SAN_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/SAN/SAN20121106v6_PctMap/SAN20121106v6_PctMap.shp")
SAR_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/SAR/SAR20140109v6_PctMapf831e47b-44f3-4954-a8e8-052b3a084322/SarasotaCounty_Pcts_051613_region.shp")
SEM_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/SEM/SEM20121106v6_PctMap/SEM20121106v6_PctMap.shp")
STJ_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/STJ/STJ20121106v6_PctMap7902e8be-3a36-4698-9e89-0b6a34bc20c1/STJ20121106v6_PctMap.shp")
STL_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/STL/STL20121106v6_PctMap/STL20121106v6_PctMap.shp")
SUW_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/SUW/SUW20121106v6_PctMap/SUW20121106v6_PctMap.shp")
UNI_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/UNI/UNI20121106v6_PctMap/UNI20121106v6_PctMap.shp")
VOL_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/VOL/VOL20160721_PctMap05578db2-b193-44b2-988d-0ba4f6c63ead/VOL20160721_PctMap.shp")
WAK_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/WAK/WAK20121106v6_PctMap/WAK20121106v6_PctMap.shp")
WAL_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/WAL/WAL20121106v6_PctMap/WAL20121106v6_PctMap.shp")
WAS_pct=gp.read_file("./raw-from-source/Records_Request/Precinct and Polling Place Files - 2012-present - last checked 7-23-2020/WAS/WAS20121106v6_PctMap/WAS20121106v6_PctMap.shp")
print(WAS_pct.head())
WAS_pct=WAS_pct.dissolve(by="pct_std")
BRE_pct=BRE_pct.dissolve(by="EPrecinct")
OSC_pct=OSC_pct.dissolve(by="PCT2014")
WAS_pct.reset_index(inplace=True)
BRE_pct.reset_index(inplace=True)
OSC_pct.reset_index(inplace=True)

  Precinct  pct_std                                           geometry
0        1  WAS0001  POLYGON ((-85.75195 30.78339, -85.75198 30.775...
1       10  WAS0010  POLYGON ((-85.57119 30.74723, -85.57122 30.746...
2      10A  WAS0010  POLYGON ((-85.61171 30.75479, -85.61021 30.754...
3       11  WAS0011  POLYGON ((-85.81292 30.54333, -85.81192 30.542...
4       12  WAS0012  POLYGON ((-85.62947 30.52597, -85.62905 30.525...


In [225]:
ALA_pct['new_county']='ALA'
BAK_pct['new_county']='BAK'
BAY_pct['new_county']='BAY'
BRA_pct['new_county']='BRA'
BRE_pct['new_county']='BRE'
BRO_pct['new_county']='BRO'
CAL_pct['new_county']='CAL'
CHA_pct['new_county']='CHA'
CIT_pct['new_county']='CIT'
CLA_pct['new_county']='CLA'
CLL_pct['new_county']='CLL'
CLM_pct['new_county']='CLM'
DES_pct['new_county']='DES'
DIX_pct['new_county']='DIX'
DUV_pct['new_county']='DUV'
ESC_pct['new_county']='ESC'
FLA_pct['new_county']='FLA'
FRA_pct['new_county']='FRA'
GLA_pct['new_county']='GLA'
GUL_pct['new_county']='GUL'
HAR_pct['new_county']='HAR'
HEN_pct['new_county']='HEN'
HER_pct['new_county']='HER'
HIL_pct['new_county']='HIL'
HOL_pct['new_county']='HOL'
IND_pct['new_county']='IND'
JEF_pct['new_county']='JEF'
LAF_pct['new_county']='LAF'
LEO_pct['new_county']='LEO'
LEV_pct['new_county']='LEV'
LIB_pct['new_county']='LIB'
MAN_pct['new_county']='MAN'
MON_pct['new_county']='MON'
MRT_pct['new_county']='MRT'
NAS_pct['new_county']='NAS'
OKA_pct['new_county']='OKA'
OKE_pct['new_county']='OKE'
ORA_pct['new_county']='ORA'
OSC_pct['new_county']='OSC'
PAL_pct['new_county']='PAL'
PAS_pct['new_county']='PAS'
POL_pct['new_county']='POL'
PUT_pct['new_county']='PUT'
SAN_pct['new_county']='SAN'
SAR_pct['new_county']='SAR'
SEM_pct['new_county']='SEM'
STJ_pct['new_county']='STJ'
STL_pct['new_county']='STL'
SUW_pct['new_county']='SUW'
UNI_pct['new_county']='UNI'
VOL_pct['new_county']='VOL'
WAK_pct['new_county']='WAK'
WAL_pct['new_county']='WAL'
WAS_pct['new_county']='WAS'

In [226]:
fl_2016_shapefiles = [ALA_pct,BAK_pct,BAY_pct,BRA_pct,BRE_pct,BRO_pct,CAL_pct,CHA_pct,CIT_pct,CLA_pct,CLL_pct,CLM_pct,
DES_pct,DIX_pct,DUV_pct,ESC_pct,FLA_pct,FRA_pct,GLA_pct,GUL_pct,HAR_pct,
HEN_pct,HER_pct,HIL_pct,HOL_pct,IND_pct,JEF_pct,LAF_pct,LEO_pct,LEV_pct,LIB_pct,MAN_pct,MON_pct,MRT_pct,NAS_pct,
OKA_pct,OKE_pct,ORA_pct,OSC_pct,PAL_pct,PAS_pct,POL_pct,PUT_pct,SAN_pct,SAR_pct,
SEM_pct,STJ_pct,STL_pct,SUW_pct,UNI_pct,VOL_pct,WAK_pct,WAL_pct,WAS_pct]

In [227]:
sum_var = 0
for i in fl_2016_shapefiles:
    sum_var+=i.shape[0]
print(sum_var)

4653


In [228]:
no_pct_std = []
filtered_list = []
for i in fl_2016_shapefiles:
    i.columns = i.columns.str.upper()
    if "PCT_STD" not in (i.columns):
        print(i.head(2))
        print("")
        print("")
        print("")
        no_pct_std.append(i)
    else:
        i_new = i[["PCT_STD","GEOMETRY","NEW_COUNTY"]]
        i_new["PCT_STD"]=i_new["PCT_STD"].str[3:]
        filtered_list.append(i_new)

filtered_list_two = []
remaining_precincts =[]
for i in no_pct_std:
    if "PRECINCT" in (i.columns):
        print(i.head(2))
        i_new = i[["PRECINCT","GEOMETRY","NEW_COUNTY"]]
        filtered_list_two.append(i_new)
    else:
        remaining_precincts.append(i)

final_pcts = []
for i in remaining_precincts:
    print(i["NEW_COUNTY"].unique())
    print(i.columns)
    print(i.head(1))
    print("")
    print("")
    if (i["NEW_COUNTY"].unique() == ["BRE"]):
        i_new = i[['EPRECINCT','GEOMETRY','NEW_COUNTY']]
        final_pcts.append(i_new)
    if (i["NEW_COUNTY"].unique() == ['OKE']):
        i_new = i[['DISTRICT','GEOMETRY','NEW_COUNTY']]
        final_pcts.append(i_new)
    if (i["NEW_COUNTY"].unique() == ['ORA']):
        i_new = i[['PRECMAY201','GEOMETRY','NEW_COUNTY']]
        final_pcts.append(i_new)
    if (i["NEW_COUNTY"].unique() == ['OSC']):
        i_new = i[['PCT2014','GEOMETRY','NEW_COUNTY']]
        final_pcts.append(i_new)
    if (i["NEW_COUNTY"].unique() == ['POL']):
        i_new = i[['DISTRICT','GEOMETRY','NEW_COUNTY']]
        final_pcts.append(i_new)
    if (i["NEW_COUNTY"].unique() == ['PUT']):
        i_new = i[['PRECINCTID','GEOMETRY','NEW_COUNTY']]
        final_pcts.append(i_new)

<ipython-input-228-749bba21088c>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i_new["PCT_STD"]=i_new["PCT_STD"].str[3:]
<ipython-input-228-749bba21088c>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i_new["PCT_STD"]=i_new["PCT_STD"].str[3:]
<ipython-input-228-749bba21088c>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

   EPRECINCT                                           GEOMETRY    PCT  CC  \
0          0  MULTIPOLYGON (((702849.618 1426823.098, 702813...    0.1   1   
1        100  MULTIPOLYGON (((705276.735 1541536.657, 705274...  100.1   1   

   CDS  CNG  CTY  HSE  PA  REC  SB  SD  SEN  SID PCTSAREAS EPCT_AREA  WRD  \
0    0    8    0   51   1    0   1   0   14    0         0      None    0   
1    0    8    0   50   1    1   1   0   14    0    100-02   Area 02    0   

      SHAPE_LENG    SHAPE_AREA NEW_COUNTY  
0  164657.690168  2.928648e+08        BRE  
1    3607.893761  5.082329e+05        BRE  



  PRECINCT                                           GEOMETRY NEW_COUNTY
0     A001  POLYGON ((-80.07938 26.30535, -80.07990 26.305...        BRO
1     A002  POLYGON ((-80.09557 26.30500, -80.09757 26.304...        BRO



  PRECINCT PRECINCTNA                                           GEOMETRY  \
0      100       None  POLYGON Z ((-82.57377 28.96227 0.00000, -82.57...   
1      101       None  P

<ipython-input-228-749bba21088c>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i_new["PCT_STD"]=i_new["PCT_STD"].str[3:]
<ipython-input-228-749bba21088c>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i_new["PCT_STD"]=i_new["PCT_STD"].str[3:]
<ipython-input-228-749bba21088c>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

  PRECINCT                                           GEOMETRY  \
0      100  POLYGON Z ((-81.98138 30.05515 0.00000, -81.98...   
1      102  POLYGON Z ((-81.89505 30.04179 0.00000, -81.89...   

              NAME NEW_COUNTY  
0  120190301021000        CLA  
1  120190301022000        CLA  
  PRECINCT                                           GEOMETRY NEW_COUNTY
0      101  POLYGON Z ((-81.37388 25.81146 0.00000, -81.37...        CLL
1      102  POLYGON Z ((-81.34397 26.04007 0.00000, -81.34...        CLL
  PRECINCT                                           GEOMETRY NEW_COUNTY
0     1001  POLYGON ((-81.72017 30.41469, -81.72088 30.414...        DUV
1     1002  POLYGON ((-81.69907 30.38242, -81.69909 30.382...        DUV
  PRECINCT                                           GEOMETRY NEW_COUNTY
0        1  POLYGON Z ((-87.26049 30.35324 0.00000, -87.26...        ESC
1      101  POLYGON Z ((-87.28370 30.53355 0.00000, -87.28...        ESC
  PRECINCT                                         

In [230]:
source_shapes_list = filtered_list + filtered_list_two + final_pcts
#print(len(source_shapes_list))
for i in source_shapes_list:
    i.columns = ["PRECINCT","GEOMETRY","COUNTY"]

source_shapes = pd.concat(source_shapes_list)



In [231]:
print(source_shapes.head())
print(source_shapes.shape)

  PRECINCT                                           GEOMETRY COUNTY
0     0001  POLYGON Z ((-82.24245 29.85246 0.00000, -82.24...    ALA
1     0002  POLYGON Z ((-82.41775 29.92248 0.00000, -82.41...    ALA
2     0003  POLYGON Z ((-82.53335 29.84801 0.00000, -82.52...    ALA
3     0004  POLYGON Z ((-82.55700 29.65461 0.00000, -82.55...    ALA
4     0005  POLYGON Z ((-82.34441 29.66672 0.00000, -82.34...    ALA
(4653, 3)


In [237]:
census_block_precincts=census_block_precincts[["Precinct","geometry","County"]]
census_block_precincts.columns=["PRECINCT","GEOMETRY","COUNTY"]

In [240]:
precinct_shapefiles = pd.concat([census_block_precincts,source_shapes])
print(precinct_shapefiles.head())

  PRECINCT                                           GEOMETRY COUNTY countypct
0        1  POLYGON ((-80.11869 25.95714, -80.11992 25.957...    DAD       NaN
1       10  POLYGON ((-80.13581 25.87200, -80.13810 25.871...    DAD       NaN
2      100  POLYGON ((-80.35118 25.80440, -80.35095 25.804...    DAD       NaN
3      101  POLYGON ((-80.15749 25.96307, -80.15965 25.963...    DAD       NaN
4      102  POLYGON ((-80.14698 25.95983, -80.14705 25.961...    DAD       NaN


In [241]:
precinct_shapefiles["PRECINCT"]=precinct_shapefiles["PRECINCT"].str.zfill(5)
precinct_shapefiles["countypct"]=precinct_shapefiles["COUNTY"]+precinct_shapefiles["PRECINCT"]

In [248]:
print(pivoted_2016.shape)
print(precinct_shapefiles.shape)

(6006, 13)
(6250, 4)


In [250]:
shapes_and_elections = pd.merge(precinct_shapefiles,pivoted_2016,on="countypct",how="outer",indicator=True)
print(shapes_and_elections["_merge"].value_counts())
shapes_and_elections[shapes_and_elections["_merge"]=="left_only"]["countypct"].to_csv("./shapefile_only.csv")
shapes_and_elections[shapes_and_elections["_merge"]=="right_only"]["countypct"].to_csv("./elections_only.csv")

both          4062
left_only     2194
right_only    1944
Name: _merge, dtype: int64


In [ ]:
#Dealt with 36 of these
for i in filtered_list:
    print(i.head(1))

In [ ]:
for i in filtered_list_two:
    print(i.head(1))

In [ ]:
print(len(filtered_list_two))
print(len(filtered_list))
print(len(remaining_precincts))